In [5]:
"Каково среднее время игры (в часах) у пользователей, которые написали отзыв во время раннего доступа (Early Access)?"

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

# Создание SparkSession и запуск Apache Spark в локальном режиме
spark = (
    SparkSession.builder
    .appName("SteamEarlyAccess")
    .master("local[*]")
    .config("spark.driver.host", "127.0.0.1")   # адрес драйвера Spark
    .config("spark.driver.bindAddress", "127.0.0.1")  # адрес привязки драйвера
    .getOrCreate()
)

file_path = "data/steam_reviews.csv"

# header=True — первая строка содержит заголовки колонок
# inferSchema=True — автоматическое определение типов данных
df = spark.read.csv(
    file_path,
    header=True,
    inferSchema=True
)

# Приведение признака Early Access из строкового типа к логическому
df = df.withColumn(
    "written_during_early_access",
    (col("written_during_early_access") == "True")
)

# Приведение времени игры к числовому типу (в минутах)
# и создание отдельной колонки playtime
df = df.withColumn(
    "playtime",
    col("`author.playtime_forever`").cast("double")
)

# Фильтрация пользователей, написавших отзыв во время Early Access
early = df.filter(col("written_during_early_access") == True)

result = early.select(
    (col("playtime") / 60).alias("hours")
).agg(
    avg("hours").alias("avg_hours")
)

print("=== SPARK ===")
result.show()

spark.stop()


=== SPARK ===
+-----------------+
|        avg_hours|
+-----------------+
|280.4050350816218|
+-----------------+

